In [ ]:
%matplotlib inline
import os
import tempfile

import scipy.stats as st

from pyabc import (ABCSMC, RV,
                   PercentileDistanceFunction, DistanceFunction, sampler)
from pyabc import Distribution as abcDis

import sys 
sys.path.append('../../')
from model_comparison.utils import *
from model_comparison.mdns import *
from model_comparison.models import PoissonModel, NegativeBinomialModel

In [ ]:
seed = 1
sample_size = 10
ntrain = int(1e6)
ntest = 200

k2 = 2.
theta2 = 1.0

k3 = 2.
theta3 = 2. 

# then the scale of the Gamma prior for the Poisson is given by
theta1 = 2.0
k1 = (k2 * theta2 * k3 * theta3) / theta1
print(k1)


model_poisson = PoissonModel(sample_size=sample_size, seed=seed, n_workers=1)
model_nb = NegativeBinomialModel(sample_size=sample_size, seed=seed, n_workers=1)
# from Gamma prior for Poisson 
prior_lam = scipy.stats.gamma(a=k1, scale=theta1)
prior_k = scipy.stats.gamma(a=k2, scale=theta2)
prior_theta = scipy.stats.gamma(a=k3, scale=theta3)

In [ ]:
# the models need to be defined in this way
def model_1(parameters): 
    x = model_poisson.gen([parameters.lam])
    return {'y': np.array([x.mean(), x.std()])}

def model_2(parameters): 
    x = model_nb.gen([[parameters.k, parameters.theta]])
    return {'y': np.array([x.mean(), x.std()])}

In [ ]:
# there is a wrapper function around scipy.stats. functions for defining the prior 
# type: name of the scipy stats function 
# kwargs: the kwards of that functions, e.g., kwargs of scipy.stats.gamma
prior1 = abcDis.from_dictionary_of_dictionaries(dict(lam={'type': 'gamma', 'kwargs': {'a':k1, 'scale': theta1}}))

prior2 = abcDis.from_dictionary_of_dictionaries(dict(k={'type': 'gamma', 'kwargs': {'a':k2, 'scale': theta2}}, 
                                                     theta={'type': 'gamma', 'kwargs': {'a':k3, 'scale': theta3}}))

In [ ]:
# We define two models, but they are identical so far
models = [model_1, model_2]

# However, our models' priors are not the same.
# Their mean differs.
parameter_priors = [prior1, prior2]

In [ ]:
# define an own distance function: mean squared distance error 
class MyDist(DistanceFunction): 
    
    def __call__(self, x, y): 
        return np.power(x['y'] - y['y'], 2).mean()        

In [ ]:
# We plug all the ABC options together
abc = ABCSMC(
    models, parameter_priors,
    MyDist(), sampler=sampler.SingleCoreSampler())

In [ ]:
# y_observed is the important piece here: our actual observation.

# set ground truth here 
# xo = model_poisson.gen([2])
xo = model_nb.gen([[2, 3]])
y_observed = [xo.mean(), xo.std()]

In [ ]:
# and we define where to store the results
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))
abc_id = abc.new(db_path, {"y": y_observed})

In [ ]:
print("ABC-SMC run ID:", abc_id)

In [ ]:
history = abc.run(minimum_epsilon=0.05, max_nr_populations=4, verbose=False)

In [ ]:
# Evaluate the model probabililties
model_probabilities = history.get_model_probabilities()
model_probabilities

In [ ]:
model_probabilities[0][3]